<a href="https://colab.research.google.com/github/gemaware/greenmovil/blob/main/ETLPlanoNetflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd

# Definir el path del archivo CSV crudo
file_path = '/content/netflix_titles.csv'

# Leer el archivo como texto, manejando la codificación a utf-8 para controlar y reemplazar
# caracteres especiales
with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
    # Leer todo el contenido del archivo
    content = f.read()

# Escribir el contenido limpio en un nuevo archivo temporal
from io import StringIO
data = StringIO(content)

netflix_data = pd.read_csv(data) #paso a pandas dataframe


# pasar a dataframe copia para limpieza
netflix_data_clean = netflix_data[['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
                                   'release_year', 'rating', 'duration', 'listed_in', 'description']].copy()


################################ TRANSFORMACIONES DE LIMPIEZA ##############################################

# Diccionario de reemplazo para caracteres mal decodificados y especiales
replace_dict = {
    'Ã©': 'é', 'Ã¡': 'á', 'Ã³': 'ó', 'Ãº': 'ú', 'Ã': 'Á', 'Ã­': 'í', 'Ãñ': 'ñ', 'Ãè': 'è', 'Ã ': 'à',
    'Ãç': 'ç', 'Ã¢': 'â', 'Ã£': 'ã', 'Ã¼': 'ü', 'Ãª': 'ê', 'Ã“': 'Ó', 'Ã‰': 'É', 'Ã': 'a', '©': 'c',
    '¬': '', '¤': '', '¡': '', '¿': '', '¶': '', '®': '', '°': '', '¦': '', '¸': '', '¨': '', '¥': '',
    '£': '', '¡': '', '«': '', '»': '', '·': '', '§': '', '¯': '', '\xa0': '', '±': '', '\xad': '',
    '\n': ' ', '&': 'and', "'": '', '/': ' ', '*': '', '~': '', '@': 'at', '.': '', '?': '', '-': ' ',
    '"': '', ':': '', '(': '', ')': '', '©': '', '¶': '', '‘': '', '%': '', '’': '', '้': '',
    '”': '', '่': '', '„': '', '\u200b': '', '์': '', '“': '', '!': '', '+': '', '–': '', '…': '',
    '#': '', '=': '', '|': '', '_': ''
}


# Función para limpiar cualquier columna de texto reemplazando caracteres mal decodificados y especiales
def clean_text(text):
    if isinstance(text, str):  # Verificamos si el valor es un string antes de aplicar el replace
        # Aplicar reemplazos de caracteres mal decodificados y especiales
        for char, replacement in replace_dict.items():
            text = text.replace(char, replacement)
        return text.strip()
    else:
        return text  # Si no es string (por ejemplo, NaN), lo dejamos como está

# Aplicar la función de limpieza de caracteres especiales y anomalos a las columnas
netflix_data_clean['title'] = netflix_data_clean['title'].apply(clean_text)
netflix_data_clean['director'] = netflix_data_clean['director'].apply(clean_text)
netflix_data_clean['cast'] = netflix_data_clean['cast'].apply(clean_text)
netflix_data_clean['country'] = netflix_data_clean['country'].apply(clean_text)
netflix_data_clean['release_year'] = netflix_data_clean['release_year'].apply(clean_text)
netflix_data_clean['rating'] = netflix_data_clean['rating'].apply(clean_text)
netflix_data_clean['duration'] = netflix_data_clean['duration'].apply(clean_text)


# Convertir la columna 'date_added' al formato DD/MM/YYYY para facilitar la analitica posteriormente
netflix_data_clean['date_added'] = pd.to_datetime(netflix_data_clean['date_added'], format='%B %d, %Y', errors='coerce').dt.strftime('%d/%m/%Y')

# Rellenar los valores nulos o vacíos en la columna 'title' con "Sin título definido"
netflix_data_clean['title'] = netflix_data_clean['title'].fillna("Sin título definido")
netflix_data_clean['title'] = netflix_data_clean['title'].apply(lambda x: "Sin título definido" if x == "" else x)

# Rellenar los valores nulos o vacíos en la columna 'director' con "Sin director definido"
netflix_data_clean['director'] = netflix_data_clean['director'].fillna("Sin director definido")
netflix_data_clean['director'] = netflix_data_clean['director'].apply(lambda x: "Sin director definido" if x == "" else x)

# Rellenar los valores nulos o vacíos en la columna 'country' con "Sin pais definido"
netflix_data_clean['country'] = netflix_data_clean['country'].fillna("Sin pais definido")
netflix_data_clean['country'] = netflix_data_clean['country'].apply(lambda x: "Sin pais definido" if x == "" else x)

# Rellenar los valores nulos o vacíos en la columna 'cast' con "Sin reparto definido"
netflix_data_clean['cast'] = netflix_data_clean['cast'].fillna("Sin resparto definido")
netflix_data_clean['cast'] = netflix_data_clean['cast'].apply(lambda x: "Sin reparto definido" if x == "" else x)

# Rellenar los valores nulos o vacíos en la columna 'listed_in' con "Sin categoria definida"
netflix_data_clean['listed_in'] = netflix_data_clean['listed_in'].fillna("Sin categoria definida")
netflix_data_clean['listed_in'] = netflix_data_clean['listed_in'].apply(lambda x: "Sin categoria definida" if x == "" else x)

# Rellenar los valores nulos o vacíos en la columna 'duration' con "Sin duracion definida"
netflix_data_clean['duration'] = netflix_data_clean['duration'].fillna("Sin duracion definida")
netflix_data_clean['duration'] = netflix_data_clean['duration'].apply(lambda x: "Sin duracion definida" if x == "" else x)

######################################################################################################################

################################### EXPORTAR EL ARCHIVO FINAL LIMPIO #######################################################
#archivo final resultado de la limpieza
#listo para procesos de analitica o extraccion hacia base de datos o data lake
netflix_data_clean.to_excel('netflix_titles_clean.xlsx', index=False)
########################################################################################################################


#################################### TRANSFORMACIONES DE FORMA #########################################################

# preparar llaves para las tablas maestras y el contenido que seran explotadas en el visualizador
def create_autoincrement_id(df, id_name):
    df[id_name] = range(1, len(df) + 1)
    return df

# Extraer los valores únicos para las tablas maestras
df_countries = pd.DataFrame(netflix_data_clean['country'].dropna().unique(), columns=['name'])
df_ratings = pd.DataFrame(netflix_data_clean['rating'].dropna().unique(), columns=['rating'])
df_categories = pd.DataFrame(list(set(cat.strip() for sublist in netflix_data_clean['listed_in'].dropna().str.split(',') for cat in sublist)), columns=['name'])
df_directors = pd.DataFrame(netflix_data_clean['director'].dropna().unique(), columns=['name'])
df_casts = pd.DataFrame(list(set(actor.strip() for sublist in netflix_data_clean['cast'].dropna().str.split(',') for actor in sublist)), columns=['name'])

# Crear IDs autoincrementales para las tablas maestras
df_countries = create_autoincrement_id(df_countries, 'country_id')
df_ratings = create_autoincrement_id(df_ratings, 'rating_id')
df_categories = create_autoincrement_id(df_categories, 'category_id')
df_directors = create_autoincrement_id(df_directors, 'director_id')
df_casts = create_autoincrement_id(df_casts, 'cast_id')

# Crear el archivo Excel para cada tabla maestra
df_countries.to_excel('Countries.xlsx', index=False)
df_ratings.to_excel('Ratings.xlsx', index=False)
df_categories.to_excel('Categories.xlsx', index=False)
df_directors.to_excel('Directors.xlsx', index=False)
df_casts.to_excel('Casts.xlsx', index=False)

# Ahora, vamos a crear la tabla "Content" relacionando con las tablas maestras

# Relacionar contenido con las tablas maestras
netflix_data_clean['country_id'] = netflix_data_clean['country'].map(df_countries.set_index('name')['country_id'])
netflix_data_clean['rating_id'] = netflix_data_clean['rating'].map(df_ratings.set_index('rating')['rating_id'])
netflix_data_clean['director_id'] = netflix_data_clean['director'].map(df_directors.set_index('name')['director_id'])

# Crear la tabla Content
df_content = netflix_data_clean[['title', 'type', 'director_id', 'country_id', 'date_added', 'release_year', 'rating_id', 'description']].copy()

# Agregar ID autoincremental para Content
df_content = create_autoincrement_id(df_content, 'content_id')

# Guardar el archivo Excel para Content
df_content.to_excel('Content.xlsx', index=False)

# Crear las tablas Movies y TV_Shows a partir de Content
df_movies = df_content[df_content['type'] == 'Movie'][['content_id']].copy()
df_movies['duration'] = netflix_data_clean['duration']

df_tv_shows = df_content[df_content['type'] == 'TV Show'][['content_id']].copy()
df_tv_shows['seasons'] = netflix_data_clean['duration']

# Agregar IDs autoincrementales para Movies y TV_Shows
df_movies = create_autoincrement_id(df_movies, 'movie_id')
df_tv_shows = create_autoincrement_id(df_tv_shows, 'show_id')

# Guardar los archivos Excel para Movies y TV_Shows
df_movies.to_excel('Movies.xlsx', index=False)
df_tv_shows.to_excel('TV_Shows.xlsx', index=False)

# Imprimir la ruta de los archivos generados
print("Archivos generados: 'Countries.xlsx', 'Ratings.xlsx', 'Categories.xlsx', 'Directors.xlsx', 'Casts.xlsx', 'Content.xlsx', 'Movies.xlsx', 'TV_Shows.xlsx'")


########################################################################################################################




Archivos generados: 'Countries.xlsx', 'Ratings.xlsx', 'Categories.xlsx', 'Directors.xlsx', 'Casts.xlsx', 'Content.xlsx', 'Movies.xlsx', 'TV_Shows.xlsx'
